<a href="https://colab.research.google.com/github/mbubesa/ML-Algo-Exploration/blob/main/Pytorch_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A busy man's intro to Pytorch


In [4]:
import torch
import numpy as np

In [2]:
a = [1, 2, 3, 4, 5]
tensor_a = torch.tensor(a)
print(tensor_a)

tensor([1, 2, 3, 4, 5])


In [5]:
# From a NumPy array
b = np.array([[1, 2, 3], [4, 5, 6]])
tensor_b = torch.tensor(b)
print(tensor_b)


tensor([[1, 2, 3],
        [4, 5, 6]])


In [6]:
# Directly
# Float Tensor of Zeros
zeros_tensor = torch.zeros(2, 3)
print(zeros_tensor)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [7]:
# Integer tensor of Ones
ones_tensor = torch.ones(3, 2)
print(ones_tensor)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [8]:
# Random Float tensor
rand_tensor = torch.rand(4, 4)
print(rand_tensor)

tensor([[0.6468, 0.3381, 0.6756, 0.0185],
        [0.6888, 0.8586, 0.6369, 0.9936],
        [0.3869, 0.0725, 0.6919, 0.0631],
        [0.4037, 0.7547, 0.3168, 0.7791]])


In [9]:
# Tensor with Specific Values
values_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(values_tensor)

tensor([[1, 2, 3],
        [4, 5, 6]])


# Tensor Properties

In [10]:
print(tensor_a.shape)
print(tensor_b.size())

torch.Size([5])
torch.Size([2, 3])


# Data Types

In [11]:
# Specify Data Type
float_tensor = torch.tensor([1, 2, 3], dtype=torch.float32)
print(float_tensor)


tensor([1., 2., 3.])


In [12]:
# Change Data Type
int_tensor = float_tensor.int()
print(int_tensor.dtype)

torch.int32


# Let's do some modeling and train a simple model

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [14]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 15254248.30it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 271283.09it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4926737.69it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5375224.54it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [15]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [19]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [20]:
# Optimizing model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)